In [ ]:
!pip install torch sentencepiece \
  accelerate \
  bitsandbytes \
  git+https://github.com/huggingface/transformers.git@15641892985b1d77acc74c9065c332cd7c3f7d7f \
  git+https://github.com/huggingface/peft.git\
  docx2txt \
  rank_bm25

In [ ]:
!pip install -U transformers

In [ ]:
import os
import docx2txt
from rank_bm25 import BM25Okapi
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

In [ ]:
MODEL_NAME = "IlyaGusev/saiga_mistral_7b"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
DEFAULT_RESPONSE_TEMPLATE = "<s>bot\n"
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."


class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        response_template=DEFAULT_RESPONSE_TEMPLATE
    ):
        self.message_template = message_template
        self.response_template = response_template
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += DEFAULT_RESPONSE_TEMPLATE
        return final_text.strip()


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto"
)
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

In [ ]:
def split_text(text, max_words_per_split):
    splits = []
    words = text.split()
    for i in range(0, len(words), max_words_per_split):
        split = ' '.join(words[i:i+max_words_per_split])
        splits.append(split)
    return splits

def read_docx_files_from_folder(folder_path):
    texts = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.docx'):
            docx_path = os.path.join(folder_path, filename)
            text = docx2txt.process(docx_path)
            texts.append(text)
    return texts

def get_top_matching_texts(folder_path, query, max_words_per_split=100, top_n=3):
    all_splits = []
    for text in read_docx_files_from_folder(folder_path):
        splits = split_text(text, max_words_per_split)
        all_splits.extend(splits)

    tokenized_texts = [text.split() for text in all_splits]

    bm25_model = BM25Okapi(tokenized_texts)

    tokenized_query = query.split(' ')

    scores = bm25_model.get_scores(tokenized_query)

    results = [(score, text) for score, text in zip(scores, all_splits)]

    sorted_results = sorted(results, reverse=True)

    top_results = sorted_results[:top_n]

    prompts = []
    for idx, (score, text) in enumerate(top_results, start=1):
        prompt = f"{text}"
        prompts.append(prompt)

    return prompts

In [ ]:
folder_path = '/content/data'
query = "Представь, что ты - специаллист по игре спортивная мафия. Что значит термин выставлять под последние руки?"
top_matching_texts = get_top_matching_texts(folder_path, query)

In [ ]:
# without RAG
inputs = [f"{query}"]
for inp in inputs:
    conversation = Conversation()
    conversation.add_user_message(inp)
    prompt = conversation.get_prompt(tokenizer)

    output = generate(model, tokenizer, prompt, generation_config)
    print(prompt)
    print(output)
    print()
    print("==============================")
    print()

In [ ]:
# with RAG
inputs = [f"{top_matching_texts[0]}, {top_matching_texts[1]}, {top_matching_texts[2]}\nuser:{query}\nbot: Вот ответ на ваш вопрос длиной не более 400 слов:"]
for inp in inputs:
    conversation = Conversation()
    conversation.add_user_message(inp)
    prompt = conversation.get_prompt(tokenizer)

    output = generate(model, tokenizer, prompt, generation_config)
    print(prompt)
    print(output)
    print()
    print("==============================")
    print()